In [1]:
import calendar
import pandas as pd
import panel as pn

from datetime import date, timedelta
from sqlalchemy import create_engine

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 7, 17)

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trade name qty target active reason market xdate'.split()

In [3]:
pd.read_sql_query('SELECT * FROM orders ORDER BY name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,4,B,AIT,18000,4.20,0,52WL,SET,2022-02-02
1,5,B,AP,10000,9.50,0,MinP,SET100,2022-05-10
2,6,B,ASK,3000,30.00,0,52WL,SET,2022-03-03
3,7,B,BANPU,12000,11.50,0,1L,SET50,2022-04-11
4,18,S,BCH,3000,21.00,0,CP1S,SET100,2022-08-31
5,8,B,BCPG,9000,10.00,0,52WL,SET100,2022-02-02
6,16,S,DIF,10000,14.60,0,CUT,SET,2022-08-10
7,19,S,GLOBAL,4500,18.50,0,C18.40,SET50,2022-02-02
8,9,B,IVL,3000,43.50,0,13WL,SET50,2022-05-31
9,10,B,KKP,3000,60.00,0,52WL,SET100,2022-04-28


### After call Ord-Log.bat which calculates spreads between target price and SET price.

In [4]:
file_name   = 'orders-log.csv'
input_file = data_path + file_name
#col_names   = ['trade','name','spreads','reason','market','qty','target_price','set_price','chg_amt','chg_pct','active']
df_log = pd.read_csv(input_file, sep=',',  index_col=None)
df_log

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate
0,B,AIT,-51,52WL,SET,18000,4.20,5.55,-0.05,-0.89%,0,2022-02-02
1,B,AP,5,MinP,SET100,10000,9.50,9.25,-0.20,-2.12%,0,2022-05-10
2,B,ASK,0,52WL,SET,3000,30.00,30.00,-0.50,-1.64%,0,2022-03-03
3,B,BANPU,-10,1L,SET50,12000,11.50,12.50,-0.10,-0.79%,0,2022-04-11
4,B,BCPG,-2,52WL,SET100,9000,10.00,10.20,-0.10,-0.97%,0,2022-02-02
5,B,IVL,-1,13WL,SET50,3000,43.50,43.75,0.00,0.00%,0,2022-05-31
6,B,KKP,-3,52WL,SET100,3000,60.00,60.75,-0.50,-0.82%,0,2022-04-28
7,B,KTC,-6,52WL,SET50,2100,52.75,54.25,0.25,+0.46%,0,2022-04-18
8,B,MCS,-4,52WL,SET,5000,10.60,11.00,-0.20,-1.79%,0,2022-09-03
9,B,RBF,5,52WL,SET100,9000,13.50,13.00,-0.30,-2.26%,0,2022-02-02


In [5]:
def calc(vals):
    spreads = vals
    if abs(spreads) < 10:
        return 1
    else:
        return 0

In [6]:
df_log['active'] = df_log['spd'].apply(calc)

In [7]:
df_log.groupby('active')['trade'].count()

active
0     6
1    14
Name: trade, dtype: int64

In [8]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

20

In [9]:
final = df_log[colt]
final

,trade,name,qty,target,active,reason,market,xdate
0,B,AIT,18000,4.20,0,52WL,SET,2022-02-02
1,B,AP,10000,9.50,1,MinP,SET100,2022-05-10
2,B,ASK,3000,30.00,1,52WL,SET,2022-03-03
3,B,BANPU,12000,11.50,0,1L,SET50,2022-04-11
4,B,BCPG,9000,10.00,1,52WL,SET100,2022-02-02
5,B,IVL,3000,43.50,1,13WL,SET50,2022-05-31
6,B,KKP,3000,60.00,1,52WL,SET100,2022-04-28
7,B,KTC,2100,52.75,1,52WL,SET50,2022-04-18
8,B,MCS,5000,10.60,1,52WL,SET,2022-09-03
9,B,RBF,9000,13.50,1,52WL,SET100,2022-02-02


In [10]:
rcds = final.values.tolist()
rcds

[['B', 'AIT', 18000, 4.2, 0, '52WL', 'SET', '2022-02-02'],
 ['B', 'AP', 10000, 9.5, 1, 'MinP', 'SET100', '2022-05-10'],
 ['B', 'ASK', 3000, 30.0, 1, '52WL', 'SET', '2022-03-03'],
 ['B', 'BANPU', 12000, 11.5, 0, '1L', 'SET50', '2022-04-11'],
 ['B', 'BCPG', 9000, 10.0, 1, '52WL', 'SET100', '2022-02-02'],
 ['B', 'IVL', 3000, 43.5, 1, '13WL', 'SET50', '2022-05-31'],
 ['B', 'KKP', 3000, 60.0, 1, '52WL', 'SET100', '2022-04-28'],
 ['B', 'KTC', 2100, 52.75, 1, '52WL', 'SET50', '2022-04-18'],
 ['B', 'MCS', 5000, 10.6, 1, '52WL', 'SET', '2022-09-03'],
 ['B', 'RBF', 9000, 13.5, 1, '52WL', 'SET100', '2022-02-02'],
 ['B', 'RCL', 3000, 35.25, 0, 'RD20pct', 'SET100', '2022-08-27'],
 ['B', 'SINGER', 3000, 35.5, 0, '52WL', 'SET100', '2022-02-02'],
 ['B', 'SNC', 9000, 16.5, 1, '1L', 'SET', '2022-03-02'],
 ['B', 'SSP', 10000, 8.75, 1, '52WL', 'SET', '2022-02-02'],
 ['B', 'TMT', 12000, 7.9, 1, 'RD20pct', 'SET', '2022-08-27'],
 ['S', 'BCH', 3000, 21.0, 1, 'CP1S', 'SET100', '2022-08-31'],
 ['S', 'DIF', 1000

In [11]:
for rcd in rcds:
    print(rcd)

['B', 'AIT', 18000, 4.2, 0, '52WL', 'SET', '2022-02-02']
['B', 'AP', 10000, 9.5, 1, 'MinP', 'SET100', '2022-05-10']
['B', 'ASK', 3000, 30.0, 1, '52WL', 'SET', '2022-03-03']
['B', 'BANPU', 12000, 11.5, 0, '1L', 'SET50', '2022-04-11']
['B', 'BCPG', 9000, 10.0, 1, '52WL', 'SET100', '2022-02-02']
['B', 'IVL', 3000, 43.5, 1, '13WL', 'SET50', '2022-05-31']
['B', 'KKP', 3000, 60.0, 1, '52WL', 'SET100', '2022-04-28']
['B', 'KTC', 2100, 52.75, 1, '52WL', 'SET50', '2022-04-18']
['B', 'MCS', 5000, 10.6, 1, '52WL', 'SET', '2022-09-03']
['B', 'RBF', 9000, 13.5, 1, '52WL', 'SET100', '2022-02-02']
['B', 'RCL', 3000, 35.25, 0, 'RD20pct', 'SET100', '2022-08-27']
['B', 'SINGER', 3000, 35.5, 0, '52WL', 'SET100', '2022-02-02']
['B', 'SNC', 9000, 16.5, 1, '1L', 'SET', '2022-03-02']
['B', 'SSP', 10000, 8.75, 1, '52WL', 'SET', '2022-02-02']
['B', 'TMT', 12000, 7.9, 1, 'RD20pct', 'SET', '2022-08-27']
['S', 'BCH', 3000, 21.0, 1, 'CP1S', 'SET100', '2022-08-31']
['S', 'DIF', 10000, 14.6, 0, 'CUT', 'SET', '2022-0

In [12]:
sql = """
INSERT INTO orders (trade, name, qty, price, active, reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""
print(sql)


INSERT INTO orders (trade, name, qty, price, active, reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)



In [13]:
for rcd in rcds:
    conlite.execute(sql, rcd)

In [14]:
sql = '''
SELECT *
FROM orders
ORDER BY trade, name'''
df_orders = pd.read_sql(sql, conlite)
df_tab = pn.widgets.Tabulator(df_orders, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=    id trade  ..., width=800)

In [15]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)